In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import seml.database as db_utils
import jkutils.mpl_latex_template as mplt

import os 
os.environ["PATH"] += os.pathsep + '/nfs/homedirs/schuchaj/texlive/2020/bin/x86_64-linux'

In [2]:
collection = 'cert_collective'
collection_train = 'train_collective'

jk_config = {
    'username': 'schuchaj',
    'password': 'LBGqsHEY',
    'host': 'fs.kdd.in.tum.de',
    'port': 27017,
    'db_name': 'schuchaj'
}

col = db_utils.get_collection(collection, mongodb_config=jk_config)
col_train = db_utils.get_collection(collection_train, mongodb_config=jk_config)

In [3]:
def find_items(dataset, model, max_rad, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    
    db_filter = {
        'config.restrictions.dataset' : dataset,
        'config.restrictions.model' : model,
        'config.max_rad' : max_rad,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att,
        'config.pf_minus_att': pf_minus_att,
    }
    if seed is not None:
        db_filter['config.seed'] = seed
    if col.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col.find(db_filter)
    
    ret = []
    
    for exp in exps:
        if exp['result'] is None:
            continue

        config = exp['config']
        cert_file_dir = exp['result']['cert_file']
        try:
            cert = torch.load(cert_file_dir)
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/cert_collective/{cert_file_dir.split("/")[-1]}')

        ret.append((exp, config, cert))
            
    return ret

In [4]:
def find_items_train(dataset, model, pf_minus_adj, pf_plus_att, pf_minus_att, seed=None):
    db_filter = {
        'config.dataset' : dataset,
        'config.model': model,
        'config.pf_plus_adj': 0,
        'config.pf_minus_adj': pf_minus_adj,
        'config.pf_plus_att': pf_plus_att
    }
    if seed is not None:
        db_filter['config.seed'] = seed

    if col_train.count_documents(db_filter) == 0:
        raise ValueError('No matches!')

    exps = col_train.find(db_filter)
    
    ret = []
    
    for exp in exps:
        config = exp['config']
        
        run_id = config['overwrite']
        db_collection = config['db_collection']
        save_dir = config['save_dir']

        try:
            cert = torch.load(f'{save_dir}/{db_collection}_{run_id}', map_location=torch.device('cpu'))
        except:
            cert = torch.load(f'/nfs/students/schuchaj/sparse_smoothing_results/train_collective/{db_collection}_{run_id}', map_location=torch.device('cpu'))

        ret.append((exp, config, cert))
            
    return ret

In [5]:
def grid_from_exp(exp, collective=True):
    cert = exp[2]
    if collective:
        return cert['collective_grid']
    else:
        idx_test = cert['idx_test']
        return ((cert['grid_base'][idx_test] > 0.5).sum(axis=0))

In [6]:
# Styling
sns.set()
pal_collective = sns.color_palette('Oranges', 5)
pal_train = sns.color_palette('Blues', 5)

In [7]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)

# Attributes
exps_collective_cora = find_items('cora_ml', 'GCN', [0, -1, -1], 0, 0.002, 0.6)
grid_collective = np.mean([grid_from_exp(exp)[0] for exp in exps_collective_cora], axis=0)
grid_collective /= grid_collective[0, 0]

grid_collective = grid_collective ** (1 / 1.25)

exps_train_cora = find_items_train('cora_ml', 'GCN', 0, 0.002, 0.6)
grid_train = np.mean([grid_from_exp(exp, collective=False) for exp in exps_train_cora], axis=0)
grid_train /= grid_train[0, 0]

grid_train = grid_train ** (1 / 1.25)

plt.pcolormesh(grid_collective, vmin=0, vmax=1, cmap='plasma', edgecolors=(0, 0, 0, 0.3))

clb = plt.colorbar(ticks=np.array([0, 0.2, 0.4, 0.6, 0.8, 1]) ** (1 / 1.25))
clb.ax.set_yticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1'])
plt.xlabel('Attribute deletions')
plt.ylabel('Attribute additions')

mplt.savefig(f'../experiments/joint/attributes_ours', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [9]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)


plt.pcolormesh(grid_train, vmin=0, vmax=1, cmap='plasma', edgecolors=(0, 0, 0, 0.3))

clb = plt.colorbar(ticks=np.array([0, 0.2, 0.4, 0.6, 0.8, 1]) ** (1 / 1.25))
clb.ax.set_yticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1'])
plt.xlabel('Attribute deletions')
plt.ylabel('Attribute additions')

mplt.savefig(f'../experiments/joint/attributes_trivial', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [10]:
# Deletions
# Attributes
exps_collective_cora = find_items('cora_ml', 'GCN', [-1, 0, -1], 0.4, 0.002, 0.6)
grid_collective = np.mean([grid_from_exp(exp)[:, 0, :] for exp in exps_collective_cora], axis=0)
grid_collective /= grid_collective[0, 0]
grid_collective = grid_collective ** (1 / 1.25)


exps_train_cora = find_items_train('cora_ml', 'GCN', 0.4, 0.002, 0.6)
grid_train = np.mean([grid_from_exp(exp, collective=False)[0, :, 0, :] for exp in exps_train_cora], axis=0)
grid_train /= grid_train[0, 0]
grid_train = grid_train ** (1 / 1.25)

In [11]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)


plt.pcolormesh(grid_collective, vmin=0, vmax=1, cmap='plasma', edgecolors=(0, 0, 0, 0.3))

clb = plt.colorbar(ticks=np.array([0, 0.2, 0.4, 0.6, 0.8, 1]) ** (1 / 1.25))
clb.ax.set_yticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1'])

plt.xlabel('Attribute deletions')
plt.ylabel('Edge deletions')

plt.tight_layout()

mplt.savefig(f'../experiments/joint/deletions_ours', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [12]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)


plt.pcolormesh(grid_train, vmin=0, vmax=1, cmap='plasma', edgecolors=(0, 0, 0, 0.3))

clb = plt.colorbar(ticks=np.array([0, 0.2, 0.4, 0.6, 0.8, 1]) ** (1 / 1.25))
clb.ax.set_yticklabels(['0', '0.2', '0.4', '0.6', '0.8', '1'])

plt.xlabel('Attribute deletions')
plt.ylabel('Edge deletions')

plt.tight_layout()


mplt.savefig(f'../experiments/joint/deletions_trivial', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})

In [13]:
mask = grid_train != 0

In [14]:
(grid_collective[mask] / grid_train[mask]).mean()

4.684845075132775

In [16]:
def contour_line(heat, level):
    y = (heat < level).argmax(0)
    all_satisfy = (heat >= level).all(0)
    y[all_satisfy] = heat.shape[0]
    if y[-1] != 0:
        y = np.concatenate((y, [0]))
        
    if (y == 0).sum() > 0:
        max_y = (y > 0).argmin() + 1
        y = y[:max_y]
    return y

In [46]:
mplt.newfig(width=0.49, textwidth_pt=397.48499)

sns.heatmap(grid_collective, cmap='YlGnBu_r', square=False)
y = contour_line(grid_collective, 0.5)
c = 'black'
plt.step(np.arange(len(y)), y, where='post', clip_on=False, zorder=3, c=c, label='Our')
y = contour_line(grid_train, 0.5)
plt.step(np.arange(len(y)), y, where='post', clip_on=False, zorder=3, linestyle='dashed', c=c, label='Na\\"ive')

     
    
plt.xlabel('Attribute deletions')
plt.ylabel('Edge deletions')
plt.ylim(0, grid_collective.shape[0])
plt.tight_layout()

plt.legend(loc='center left',
          frameon=False,
          handlelength=1.5,
          )
# plt.legend(loc="upper left", framealpha=0.5)

mplt.savefig(f'../experiments/joint/deletions_both', format='pgf', preview='png', dpi=200, tight={'pad': 0.5})